In [132]:
from datetime import datetime, timedelta
import glob
import time
import numpy as np
import talib

# from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly as py
import cufflinks as cf
import pandas as pd
import plotly.tools as tls
import plotly.graph_objs as go

import plotly.figure_factory as ff
init_notebook_mode(connected=True)

import pyodbc
from sqlalchemy import create_engine


#  todo driver 修正
def conn():
    return pyodbc.connect(
        'DRIVER={ODBC Driver 13 for SQL Server};' + 'SERVER=localhost;' +
        'PORT=1443;' + 'DATABASE=StockDB;' + 'UID=sa;' + 'PWD=admin;')


# Create the sqlalchemy engine using the pyodbc connection
engine = create_engine(
    "mssql+pyodbc://?driver=ODBC+Driver+13?charset=utf8", creator=conn)
con = engine.connect()
con.execute("SET LANGUAGE us_english; set dateformat ymd;")
con.close()

In [133]:
query_num = '2330'

In [134]:
cmd = """SELECT
      [date],
      [close_price],
      [open_price]
      FROM [dbo].[TECH_M] WHERE stock = \'{}\' ORDER BY date DESC"""

tech_m_df = pd.read_sql_query( cmd.format(query_num), engine )

tech_m_df[ 'date' ] = pd.to_datetime( tech_m_df[ 'date' ], format='%Y-%m-%d' )

# tech_m_df

In [135]:
import warnings

cmd = """SELECT
      [date]
      ,[Month_Revenue]
      ,[Last_Month_Revenue]
      ,[Last_Year_Revenue]
      ,[Last_Month_Ratio]
      ,[Last_Year_Ration]
      ,[Month_Acc_Revenue]
      ,[Last_Year_Acc_Revenue]
      ,[ration]
  FROM [dbo].[REVENUE] WHERE stock = \'{}\' ORDER BY date DESC"""

revenue_df = pd.read_sql_query( cmd.format(query_num), engine )

revenue_df[ 'date' ] = pd.to_datetime( revenue_df[ 'date' ], format='%Y-%m-%d' )

revenue_df[ 'history_revenue_03' ] = revenue_df[ 'Month_Revenue' ].shift( 12*1 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*2 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*3 )

revenue_df[ 'history_revenue_03' ] = revenue_df[ 'history_revenue_03' ].shift( -12*4 )

revenue_df[ 'history_revenue_03' ] = revenue_df[ 'history_revenue_03' ] / 3

revenue_df[ 'history_revenue_03%' ] = ( revenue_df[ 'Month_Revenue' ] - revenue_df[ 'history_revenue_03' ] ) / revenue_df[ 'history_revenue_03' ] * 100 
# -------------------------

revenue_df[ 'history_revenue_05' ] = revenue_df[ 'Month_Revenue' ].shift( 12*1 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*2 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*3 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*4 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*5 )

revenue_df[ 'history_revenue_05' ] = revenue_df[ 'history_revenue_05' ].shift( -12*6 )

revenue_df[ 'history_revenue_05' ] = revenue_df[ 'history_revenue_05' ] / 5 

revenue_df[ 'history_revenue_05%' ] = ( revenue_df[ 'Month_Revenue' ] - revenue_df[ 'history_revenue_05' ] ) / revenue_df[ 'history_revenue_05' ] * 100

# -------------------------

revenue_df[ 'history_revenue_08' ] = revenue_df[ 'Month_Revenue' ].shift( 12*1 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*2 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*3 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*4 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*5 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*6 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*7 ) + \
                                     revenue_df[ 'Month_Revenue' ].shift( 12*8 )                        
                        
revenue_df[ 'history_revenue_08' ] = revenue_df[ 'history_revenue_08' ].shift( -12*9 )

revenue_df[ 'history_revenue_08' ] = revenue_df[ 'history_revenue_08' ] / 8 

revenue_df[ 'history_revenue_08%' ] = ( revenue_df[ 'Month_Revenue' ] - revenue_df[ 'history_revenue_08' ] ) / revenue_df[ 'history_revenue_08' ] * 100

revenue_df[ [ 'date', 'history_revenue_03%', 'history_revenue_05%', 'history_revenue_08%' ] ].head( 48 )

# -------------------------

revenue_df[ 'history_revenue_01' ] = revenue_df[ 'Month_Revenue' ].shift( 12*1 )

revenue_df[ 'history_revenue_01' ] = revenue_df[ 'history_revenue_01' ].shift( -12*2 )

revenue_df[ 'history_revenue_01%' ] = ( revenue_df[ 'Month_Revenue' ] - revenue_df[ 'history_revenue_01' ] ) / revenue_df[ 'history_revenue_01' ] * 100

# -------------------------

warnings.filterwarnings( "ignore", category=RuntimeWarning )

revenue_df[ 'history_revenue_01%' ] = revenue_df[ 'history_revenue_01%' ].round( 1 )

revenue_df[ 'history_revenue_03%' ] = revenue_df[ 'history_revenue_03%' ].round( 1 )

revenue_df[ 'history_revenue_05%' ] = revenue_df[ 'history_revenue_05%' ].round( 1 )

revenue_df[ 'history_revenue_08%' ] = revenue_df[ 'history_revenue_08%' ].round( 1 )


# revenue_df[ [ 'date', 'history_revenue_01%', 'history_revenue_03%', 'history_revenue_05%', 'history_revenue_08%' ] ].head( 48 )

In [136]:
revenue_df.sort_values( by = 'date', inplace = True )

tech_m_df.sort_values( by = 'date', inplace = True )

result = pd.merge_asof( revenue_df, tech_m_df, on='date', direction='nearest' )

result = result[ ::-1 ]

# result

In [137]:
trace1 = go.Scatter(
    x = result[ 'date' ],
    y = result[ 'history_revenue_01%' ],
    mode = 'lines',
    name = '較前年當月營收成長%',
    line = dict( width = 1 )
)

trace2 = go.Scatter(
    x = result[ 'date' ],
    y = result[ 'history_revenue_03%' ],
    mode = 'lines',
    name = '較前3年當月營收成長%',
    line = dict( width = 1 )
)

trace3 = go.Scatter(
    x = result[ 'date' ],
    y = result[ 'history_revenue_05%' ],
    mode = 'lines',
    name = '較前5年當月營收成長%',
    line = dict( width = 1 )
)

trace4 = go.Scatter(
    x = result[ 'date' ],
    y = result[ 'history_revenue_08%' ],
    mode = 'lines',
    name = '較前8年當月營收成長%',
    line = dict( width = 1 )
)

trace5 = go.Scatter(
    x = result[ 'date' ],
    y = result[ 'open_price' ],
    mode = 'lines',
    name = 'open_price',
    line = dict( width = 2 ),
    yaxis='y2'
)

data = [ trace1, trace2, trace3, trace4, trace5 ]

layout = go.Layout(
    
    title='{} 營收'.format( query_num ),
    
    yaxis=dict(
        title='營收成長'
    ),
    
    yaxis2 = dict(
        
        title='股價',
        
        overlaying='y',
        side='right'
    )
)

fig = go.Figure( data=data, layout=layout )

py.offline.iplot( fig, filename='ohlc_datetime' )

In [138]:
cmd = """SELECT
      [date],
      [open_price],
      [high_price],
      [low_price],
      [close_price],
      [volume],
      [ma3],
      [ma10], 
      [ma60],
      [ma120],
      [ma240] FROM [dbo].[TECH_D] WHERE stock = \'{}\' ORDER BY date DESC"""

tech_d_df = pd.read_sql_query(cmd.format(query_num), engine)

# tech_d_df.head( )

tech_d_df['date'] = pd.to_datetime(tech_d_df['date'])

mask = (tech_d_df['date'] > '2017-1-1')

tech_d_df = tech_d_df.loc[mask]

trace0 = go.Candlestick( x   = tech_d_df[ 'date' ],
                       open  = tech_d_df[ 'open_price' ],
                       high  = tech_d_df[ 'high_price' ],
                       low   = tech_d_df[ 'low_price' ],
                       close = tech_d_df[ 'close_price' ],
                       name = 'K線',
                       line = dict( width = 1 ) )

trace1 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma3' ],
    mode = 'lines',
    name = '3日均線',
    line = dict( width = 1 )
)

trace2 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma10' ],
    mode = 'lines',
    name = '10日均線',
    line = dict( width = 1 )
)

trace3 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma60' ],
    mode = 'lines',
    name = '60日均線',
    line = dict( width = 1, color='rgb(82, 82, 82)' )
)

trace4 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma120' ],
    mode = 'lines',
    name = '120日均線',
    line = dict( width = 1 )
)

trace5 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'ma240' ],
    mode = 'lines',
    name = '240日均線',
    line = dict( width = 1 )
)

data = [ trace0, trace1, trace2, trace3, trace4, trace5 ]

py.offline.iplot( data, filename='ohlc_datetime')

# layout = {
#     'title': query_num,
#     'yaxis': { 'title': '股價' } }
# fig = dict( data = [trace], layout = layout )
# py.offline.iplot( fig, filename='ohlc_datetime')

In [139]:
C = np.array( tech_d_df[ 'volume' ], dtype = float, ndmin = 1 )

for val in [ 5, 10, 20 ]:
    tech_d_df[ 'MA' + str( val ) ] = talib.SMA( C, val )

trace1 = go.Bar(
    
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'volume' ],
    name = '成交量',
    
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    
    opacity=0.6
)

trace2 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'MA5' ],
    mode = 'lines',
    name = '成交均線5',
    line = dict( width = 1 )
)

trace3 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'MA10' ],
    mode = 'lines',
    name = '成交均線10',
    line = dict( width = 1 )
)

trace4 = go.Scatter(
    x = tech_d_df[ 'date' ],
    y = tech_d_df[ 'MA20' ],
    mode = 'lines',
    name = '成交均線20',
    line = dict( width = 1 )
)

data = [ trace1, trace2, trace3, trace4 ]

py.offline.iplot( data, filename='ohlc_datetime')

In [140]:
# 外資
# 投信
# 自營商

cmd = """SELECT [stock]
      ,[date]
      ,[foreign_sell]
      ,[investment_sell]
      ,[dealer_sell]
      ,[single_day_sell]
      ,[foreign_estimate]
      ,[investment_estimate]
      ,[dealer_estimate]
      ,[single_day_estimate]
      ,[foreign_ratio]
      ,[investment_ratio]
  FROM [dbo].[INVESTORS]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
investors_df = pd.read_sql_query( cmd.format( query_num ), engine )

color = np.array( ['rgb(255,255,255)'] * investors_df[ 'foreign_sell' ].shape[ 0 ] )
color[ investors_df[ 'foreign_sell' ] < 0 ] = '#5fba7d'
color[ investors_df[ 'foreign_sell' ] > 0 ] = '#d65f5f'

trace1 = go.Bar(
    
    x = investors_df[ 'date' ],
    y = investors_df[ 'foreign_sell' ],
    
    name = '外資',
    
    marker=dict(
        color= color.tolist( ),
        line = dict(
            color = color.tolist( ),
            width=1,
        )
    ),
    
    opacity = 0.6
)

trace2 = go.Scatter(
    x = investors_df[ 'date' ],
    y = investors_df[ 'foreign_estimate' ],
    mode = 'lines',
    name = '外資累計',
    line = dict( width = 1 ),
    yaxis='y2'
)

data = [ trace1, trace2 ]

layout = go.Layout(
    
    title='外資買賣超',
    
    yaxis=dict(
        title='單日買賣量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

# table = ff.create_table( investors_df.head( ) )
# py.offline.iplot( table, filename='jupyter-table1' )
# investors_df.head( )

In [141]:
color = np.array( ['rgb(255,255,255)'] * investors_df[ 'investment_sell' ].shape[ 0 ] )
color[ investors_df[ 'investment_sell' ] < 0 ] = '#5fba7d'
color[ investors_df[ 'investment_sell' ] > 0 ] = '#d65f5f'

trace1 = go.Bar(
    
    x = investors_df[ 'date' ],
    y = investors_df[ 'investment_sell' ],
    
    name = '投信',
    
    marker=dict(
        color= color.tolist( ),
        line = dict(
            color = color.tolist( ),
            width=1,
        )
    ),
    
    opacity = 0.6
)

trace2 = go.Scatter(
    x = investors_df[ 'date' ],
    y = investors_df[ 'investment_estimate' ],
    mode = 'lines',
    name = '投信累計',
    line = dict( width = 1 ),
    yaxis='y2'
)

data = [ trace1, trace2 ]

layout = go.Layout(
    
    title='投信買賣超',
    
    yaxis=dict(
        title='單日買賣量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

In [142]:
color = np.array( ['rgb(255,255,255)'] * investors_df[ 'dealer_sell' ].shape[ 0 ] )
color[ investors_df[ 'dealer_sell' ] < 0 ] = '#5fba7d'
color[ investors_df[ 'dealer_sell' ] > 0 ] = '#d65f5f'

trace1 = go.Bar(
    
    x = investors_df[ 'date' ],
    y = investors_df[ 'dealer_sell' ],
    
    name = '自營商',
    
    marker=dict(
        color= color.tolist( ),
        line = dict(
            color = color.tolist( ),
            width=1,
        )
    ),
    
    opacity = 0.6
)

trace2 = go.Scatter(
    x = investors_df[ 'date' ],
    y = investors_df[ 'dealer_estimate' ],
    mode = 'lines',
    name = '自營商累計',
    line = dict( width = 1 ),
    yaxis='y2'
)

data = [ trace1, trace2 ]

layout = go.Layout(
    
    title='自營商買賣超',
    
    yaxis=dict(
        title='單日買賣量'
    ),
    
    yaxis2 = dict(
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)

py.offline.iplot( fig, filename='ohlc_datetime' )

In [143]:
cmd = """SELECT [stock]
      ,[date]
      ,[Financing_Buy]
      ,[Financing_Sell]
      ,[Financing_PayOff]
      ,[Financing_Over]
      ,[Financing_Increase]
      ,[Financing_Limit]
      ,[Financing_Use]
      ,[Margin_Sell]
      ,[Margin_Buy]
      ,[Margin_PayOff]
      ,[Margin_Over]
      ,[Margin_Increase]
      ,[Margin_Ratio]
      ,[Margin_Offset]
  FROM [dbo].[MARGIN]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
margin_df = pd.read_sql_query( cmd.format( query_num ), engine )

color = np.array( ['rgb(255,255,255)'] * margin_df[ 'Financing_Increase' ].shape[ 0 ] )
color[ margin_df[ 'Financing_Increase' ] < 0 ] = '#5fba7d'
color[ margin_df[ 'Financing_Increase' ] > 0 ] = '#d65f5f'

trace1 = go.Bar(
    
    x = margin_df[ 'date' ],
    y = margin_df[ 'Financing_Increase' ],
    
    name = '融資差額',
    
    marker=dict(
        color= color.tolist( ),
        line = dict(
            color = color.tolist( ),
            width=1,
        )
    ),
    
    opacity = 0.6
)

trace2 = go.Scatter(
    
    x = margin_df[ 'date' ],
    y = margin_df[ 'Financing_Over' ],
    mode = 'lines',
    name = '融資張數',
    line = dict( width = 1 ),
    yaxis='y2'
)

data = [ trace1, trace2 ]

layout = go.Layout(
    
    title='融資',
    
    yaxis=dict(
        title='單日買量'
    ),
    
    yaxis2 = dict(
        
        title='累計買賣量',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot( fig, filename='ohlc_datetime' )

# margin_df.head()

In [144]:
color = np.array(['rgb(255,255,255)'] * margin_df['Margin_Increase'].shape[0])
color[margin_df['Margin_Increase'] < 0] = '#5fba7d'
color[margin_df['Margin_Increase'] > 0] = '#d65f5f'

trace1 = go.Bar(
    x=margin_df['date'],
    y=margin_df['Margin_Increase'],
    name='融卷差額',
    marker=dict(
        color=color.tolist(), line=dict(
            color=color.tolist(),
            width=1,
        )),
    opacity=0.6)

trace2 = go.Scatter(
    x=margin_df['date'],
    y=margin_df['Margin_Over'],
    mode='lines',
    name='融卷張數',
    line=dict(width=1),
    yaxis='y2')

data = [trace1, trace2]

layout = go.Layout(
    title='融卷',
    yaxis=dict(title='單日賣量'),
    yaxis2=dict(
        title='累計買賣量',
        titlefont=dict(color='rgb(148, 103, 189)'),
        tickfont=dict(color='rgb(148, 103, 189)'),
        overlaying='y',
        side='right'))

fig = go.Figure(data=data, layout=layout)

py.offline.iplot(fig, filename='ohlc_datetime')

In [145]:
cmd = """SELECT [stock]
      ,[date]
      ,[lend_over]
      ,[lend_chang]
      ,[lend_return]
      ,[lend_diff]
      ,[sell_over]
      ,[sell_day]
      ,[sell_day_return]
      ,[sell_day_diff]
      ,[sell_day_limit]
  FROM [dbo].[LEND]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
lend_df = pd.read_sql_query( cmd.format( query_num ), engine )

lend_df.head().style

,stock,date,lend_over,lend_chang,lend_return,lend_diff,sell_over,sell_day,sell_day_return,sell_day_diff,sell_day_limit
0,2330,2018-03-28,360805000,1633000,4680000,-3047000,16736000,454000,0,454000,11981284
1,2330,2018-03-27,363852000,8047000,3051000,4996000,16282000,0,2546000,-2546000,12147583
2,2330,2018-03-26,358856000,8123000,2835000,5288000,18828000,0,0,0,12093657
3,2330,2018-03-23,353568000,26749000,13700000,13049000,18828000,93000,0,93000,12007782
4,2330,2018-03-22,340519000,11047000,50000,10997000,18735000,868000,0,868000,11936116


In [146]:
# u'當月營收', 
# u'上月營收', 
# u'去年當月營收', 
# u'上月比較增減(%)', 
# u'去年同月增減(%)', 
# u'當月累計營收', 
# u'去年累計營收', 
# u'前期比較增減(%)' 

cmd = """SELECT [stock]
      ,[date]
      ,[Month_Revenue]
      ,[Last_Month_Revenue]
      ,[Last_Year_Revenue]
      ,[Last_Month_Ratio]
      ,[Last_Year_Ration]
      ,[Month_Acc_Revenue]
      ,[Last_Year_Acc_Revenue]
      ,[ration]  
  FROM [dbo].[REVENUE]
  WHERE stock = \'{}\' ORDER BY date DESC"""
  
revenue_df = pd.read_sql_query( cmd.format( query_num ), engine )

# revenue_df.head()

In [147]:
fig = tls.make_subplots( rows=8, cols=1, shared_xaxes=False )
#                         subplot_titles=( '集中度', 'K線', '主力買賣超', '主力買賣超') )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Month_Revenue'], 
                   'type': 'scatter', 
                   'name': '當月營收' }, 1, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Month_Revenue'], 
                   'type': 'scatter', 
                   'name': '上月營收' }, 2, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Year_Revenue'], 
                   'type': 'scatter', 
                   'name': '去年當月營收' }, 3, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Month_Ratio'], 
                   'type': 'scatter', 
                   'name': '上月比較增減%' }, 4, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Year_Ration'], 
                   'type': 'scatter', 
                   'name': '去年同月增減%' }, 5, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Month_Acc_Revenue'], 
                   'type': 'scatter', 
                   'name': '當月累計營收' }, 6, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['Last_Year_Acc_Revenue'], 
                   'type': 'scatter', 
                   'name': '去年累計營收' }, 7, 1 )

fig.append_trace( {'x': revenue_df['date'], 
                   'y': revenue_df['ration'], 
                   'type': 'scatter', 
                   'name': '前期比較增減' }, 8, 1 )

py.offline.iplot( fig, filename= '月營收' )

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]
[ (7,1) x7,y7 ]
[ (8,1) x8,y8 ]



In [148]:
path = r'C:\workspace\stock\01_Day process\券商分點\籌碼集中暫存.csv'

csv_df = pd.read_csv(
    path,
    sep=',',
    encoding='utf8',
    false_values='NA',
    dtype={
        '股號': str,
        '日期': str
    },
)

query_num = '2317'

filename = '{}_籌碼集中.html'.format(query_num)

csv_df = csv_df[ csv_df['股號'] == query_num ]

csv_df = csv_df[ [ '日期', '收盤', '01天集中%', '03天集中%', '05天集中%', 
                   '10天集中%', '20天集中%', '60天集中%' ] ]

csv_df = csv_df[ 0:60 ] 

csv_df = csv_df.set_index('日期')

csv_df.style.bar( subset=[ '01天集中%', '03天集中%', 
                           '05天集中%', '10天集中%', '20天集中%',
                           '60天集中%' ],
                  align='zero', 
                  color=[ '#5fba7d', '#d65f5f' ],
                  width=50 )

,收盤,01天集中%,03天集中%,05天集中%,10天集中%,20天集中%,60天集中%
日期,,,,,,,
2018/05/14,89,22.6,24.5,17.9,-0,-11.6,-17.7
2018/05/11,85,32.2,12.1,7.4,-10.1,-17.2,-19.5
2018/05/10,82.9,18.6,-0.9,-6.9,-17.6,-21.6,-19.9
2018/05/09,81.8,-24.7,-8.6,-24.5,-21.3,-23.2,nan
2018/05/08,82.6,10,-7,-26.9,-21.9,-24,nan
2018/05/04,80.4,-23.1,-38.9,-28.4,-26.8,-26.2,nan
2018/05/03,80.1,-52.5,-33.2,-26.7,-23.9,-27.3,nan
2018/04/30,82.9,-1.1,-11,-19.4,-20.4,-23.9,nan
2018/04/27,81.5,-15.7,-19.8,-24.9,-21.5,-24.7,nan


In [165]:
path = r'C:\workspace\stock\01_Day process\券商分點\籌碼集中暫存.csv'

csv_df = pd.read_csv(
    path,
    sep=',',
    encoding='utf8',
    false_values='NA',
    dtype={
        '股號': str,
        '日期': str
    },
)

query_num = '2317'

filename = '{}_籌碼集中.html'.format(query_num)

csv_df = csv_df[csv_df['股號'] == query_num]

csv_df = csv_df[[
    '日期', '收盤', '01天集中%', '03天集中%', '05天集中%', '10天集中%', '20天集中%', '60天集中%'
]]

csv_df = csv_df[0:60]

csv_df = csv_df.set_index('日期')


def high_vals(val, hi, mid, low):

    if val >= hi:
        return 'background-color: #d65f5f'
    elif val >= mid:
        return 'background-color: rgb( 250, 215, 80 )'
    elif val >= low:
        return 'background-color: rgb( 158, 202, 225 )'
    else:
        return ''


obj = csv_df.style.applymap(
    high_vals, subset=['01天集中%'], hi=20, mid=15, low=12)

obj.applymap(high_vals, subset=['03天集中%'], hi=15, mid=12, low=10)

obj.applymap(high_vals, subset=['05天集中%'], hi=15, mid=12, low=10)

obj.applymap(high_vals, subset=['10天集中%'], hi=12, mid=10, low=8)

obj.applymap(high_vals, subset=['20天集中%'], hi=10, mid=8, low=6)

obj.applymap(high_vals, subset=['60天集中%'], hi=8, mid=6, low=100)

,收盤,01天集中%,03天集中%,05天集中%,10天集中%,20天集中%,60天集中%
日期,,,,,,,
2018/05/14,89,22.6,24.5,17.9,-0,-11.6,-17.7
2018/05/11,85,32.2,12.1,7.4,-10.1,-17.2,-19.5
2018/05/10,82.9,18.6,-0.9,-6.9,-17.6,-21.6,-19.9
2018/05/09,81.8,-24.7,-8.6,-24.5,-21.3,-23.2,nan
2018/05/08,82.6,10,-7,-26.9,-21.9,-24,nan
2018/05/04,80.4,-23.1,-38.9,-28.4,-26.8,-26.2,nan
2018/05/03,80.1,-52.5,-33.2,-26.7,-23.9,-27.3,nan
2018/04/30,82.9,-1.1,-11,-19.4,-20.4,-23.9,nan
2018/04/27,81.5,-15.7,-19.8,-24.9,-21.5,-24.7,nan


In [150]:

path = r'C:\workspace\stock\01_Day process\券商分點\籌碼集中暫存.csv'

csv_df = pd.read_csv(
    path,
    sep=',',
    encoding='utf8',
    false_values='NA',
    dtype={
        '股號': str,
        '日期': str
    },
)

filename = '{}_籌碼集中.html'.format(query_num)

csv_df = csv_df[csv_df['股號'] == query_num]
csv_df = csv_df[::-1]

# hover_text = df.apply( lambda r: '<br>'.join( [ '{}: {}'.format( c, r[c] )
#                                             for c in df.columns ] ), axis=1 )

fig = tls.make_subplots(rows=2, cols=1, shared_xaxes=True)
#                         subplot_titles=( '集中度', 'K線', '主力買賣超', '主力買賣超') )

color = np.array(['rgb(255,255,255)'] * csv_df['01天主力買賣超(張)'].shape[0])
color[csv_df['01天主力買賣超(張)'] < 0] = '#5fba7d'
color[csv_df['01天主力買賣超(張)'] > 0] = '#d65f5f'

fig.append_trace({
    'x': csv_df['日期'],
    'y': csv_df['01天集中%'],
    'type': 'scatter',
    'name': '01天集中%'
}, 1, 1)

fig.append_trace({
    'x': csv_df['日期'],
    'y': csv_df['05天集中%'],
    'type': 'scatter',
    'name': '05天集中%'
}, 1, 1)

fig.append_trace(
    {
        'x': csv_df['日期'],
        'y': csv_df['01天主力買賣超(張)'],
        'type': 'bar',
        'name': '張',
        'marker': dict(color=color.tolist()),
    },
    2,
    1)

py.offline.iplot(fig, filename=filename)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



In [151]:
cmd = """SELECT [stock]
      ,[date]
      ,[Share_Rating_People_1_999]
      ,[Share_Rating_Unit_1_999]
      ,[Share_Rating_Proportion_1_999]
      ,[Share_Rating_People_1000_5000]
      ,[Share_Rating_Unit_1000_5000]
      ,[Share_Rating_Proportion_1000_5000]
      ,[Share_Rating_People_5001_10000]
      ,[Share_Rating_Unit_5001_10000]
      ,[Share_Rating_Proportion_5001_10000]
      ,[Share_Rating_People_10001_15000]
      ,[Share_Rating_Unit_10001_15000]
      ,[Share_Rating_Proportion_10001_15000]
      ,[Share_Rating_People_15001_20000]
      ,[Share_Rating_Unit_15001_20000]
      ,[Share_Rating_Proportion_15001_20000]
      ,[Share_Rating_People_20001_30000]
      ,[Share_Rating_Unit_20001_30000]
      ,[Share_Rating_Proportion_20001_30000]
      ,[Share_Rating_People_30001_40000]
      ,[Share_Rating_Unit_30001_40000]
      ,[Share_Rating_Proportion_30001_40000]
      ,[Share_Rating_People_40001_50000]
      ,[Share_Rating_Unit_40001_50000]
      ,[Share_Rating_Proportion_40001_50000]
      ,[Share_Rating_People_50001_100000]
      ,[Share_Rating_Unit_50001_100000]
      ,[Share_Rating_Proportion_50001_100000]
      ,[Share_Rating_People_100001_200000]
      ,[Share_Rating_Unit_100001_200000]
      ,[Share_Rating_Proportion_100001_200000]
      ,[Share_Rating_People_200001_400000]
      ,[Share_Rating_Unit_200001_400000]
      ,[Share_Rating_Proportion_200001_400000]
      ,[Share_Rating_People_400001_600000]
      ,[Share_Rating_Unit_400001_600000]
      ,[Share_Rating_Proportion_400001_600000]
      ,[Share_Rating_People_600001_800000]
      ,[Share_Rating_Unit_600001_800000]
      ,[Share_Rating_Proportion_600001_800000]
      ,[Share_Rating_People_800001_1000000]
      ,[Share_Rating_Unit_800001_1000000]
      ,[Share_Rating_Proportion_800001_1000000]
      ,[Share_Rating_People_Up_1000001]
      ,[Share_Rating_Unit_Up_1000001]
      ,[Share_Rating_Proportion_Up_1000001]
  FROM [dbo].[TDCC]
  WHERE stock = \'{}\' ORDER BY date DESC"""

tdcc_df = pd.read_sql_query(cmd.format(query_num), engine)

tdcc_df.head()

tdcc_df[ '散戶持股比例100張以下' ] = tdcc_df[ 'Share_Rating_Proportion_1_999' ] + \
tdcc_df['Share_Rating_Proportion_1000_5000']  + \
tdcc_df['Share_Rating_Proportion_5001_10000'] + \
tdcc_df['Share_Rating_Proportion_10001_15000'] + \
tdcc_df['Share_Rating_Proportion_15001_20000'] +\
tdcc_df['Share_Rating_Proportion_20001_30000'] +\
tdcc_df['Share_Rating_Proportion_30001_40000'] +\
tdcc_df['Share_Rating_Proportion_40001_50000'] +\
tdcc_df['Share_Rating_Proportion_50001_100000']

tdcc_df[
    '散戶持股比例200張以下'] = tdcc_df['散戶持股比例100張以下'] + tdcc_df['Share_Rating_Proportion_100001_200000']

tdcc_df['散戶持股比例400張以下'] = tdcc_df['散戶持股比例200張以下'] + \
                                 tdcc_df[ 'Share_Rating_Proportion_200001_400000']

tdcc_df['散戶持股比例600張以下'] = tdcc_df['散戶持股比例400張以下'] + \
                                 tdcc_df[ 'Share_Rating_Proportion_400001_600000']

tdcc_df['散戶持股比例800張以下'] = tdcc_df['散戶持股比例600張以下'] + \
                                 tdcc_df[ 'Share_Rating_Proportion_600001_800000']

tdcc_df['大戶持股比例400張以上'] = tdcc_df['Share_Rating_Proportion_400001_600000']  + \
tdcc_df['Share_Rating_Proportion_600001_800000']  + \
tdcc_df['Share_Rating_Proportion_800001_1000000'] + \
tdcc_df['Share_Rating_Proportion_Up_1000001']

tdcc_df['大戶持股比例600張以上'] = tdcc_df['大戶持股比例400張以上'] - \
                                  tdcc_df['Share_Rating_Proportion_400001_600000']

tdcc_df['大戶持股比例800張以上'] = tdcc_df['大戶持股比例600張以上'] - \
                                 tdcc_df['Share_Rating_Proportion_600001_800000']

tdcc_df['大戶持股比例1000張以上'] = tdcc_df['大戶持股比例800張以上'] - \
                                 tdcc_df['Share_Rating_Proportion_800001_1000000']

tdcc_df['date'] = pd.to_datetime(tdcc_df['date'])

tdcc_df = pd.merge(tdcc_df, tech_d_df, on='date', how='right')

tdcc_df = tdcc_df[tdcc_df['大戶持股比例1000張以上'].notnull()]

In [152]:
trace1 = go.Scatter(x=tdcc_df['date'], y=tdcc_df['大戶持股比例1000張以上'], name='大戶持股')

trace2 = go.Scatter(
    x=tdcc_df['date'], y=tdcc_df['散戶持股比例400張以下'], name='散戶持股', yaxis='y2')

trace3 = go.Scatter(
    x=tdcc_df['date'], y=tdcc_df['close_price'], name='股價', yaxis='y3')

data = [trace1, trace2, trace3]

layout = go.Layout(
    title='{}持股比例'.format(query_num),
    width=800,
    xaxis=dict(domain=[0.09, 1]),
    yaxis=dict(
        title='比例1000張以上',
        titlefont=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4')),
    yaxis2=dict(
        title='比例400張以下',
        titlefont=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        anchor='free',
        overlaying='y',
        side='left',
        position=0),
    yaxis3=dict(
        title='股價',
        titlefont=dict(color='#d62728'),
        tickfont=dict(color='#d62728'),
        overlaying='y',
        anchor='x',
        side='right'),
)

fig = go.Figure(data=data, layout=layout)

py.offline.iplot(fig, filename='{} 持股比例'.format(query_num))

# tech_d_df.head( 30 )

In [153]:
py.offline.plot( fig, filename = '鴻海周集保.html' )

'file://C:\\workspace\\stock\\gui\\鴻海周集保.html'